In [ ]:
#default_exp vision.widgets

In [ ]:
#export
from fastai2.torch_basics import *
from fastai2.data.all import *
from fastai2.vision.core import *
from ipywidgets import HBox,VBox,widgets,Button,Checkbox,Dropdown,Layout,Box,Output,Label

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
_all_ = ['HBox','VBox','widgets','Button','Checkbox','Dropdown','Layout','Box']

# Vision widgets

> ipywidgets for images

In [ ]:
#export
@patch
def __getitem__(self:Box, i): return self.children[i]

In [ ]:
#export
def _open_thumb(fn, h, w): return Image.open(fn).to_thumb(h, w).convert('RGBA')

In [ ]:
#export
class ImagesCleaner:
    "A widget that displays all images in `fns` along with a `Dropdown`"
    def __init__(self, opts=(), height=128, width=256, max_n=30):
        store_attr(self, 'opts,height,width,max_n')
        box_layout = dict(width='100%', height='', overflow='scroll hidden', flex_flow='row', display='flex')
        self.widget = Box(children=[], layout=box_layout)

    def set_fns(self, fns):
        self.fns = L(fns)[:self.max_n]
        img_layout = dict(height=f'{self.height}px', object_fit='contain')
        ims = parallel(_open_thumb, self.fns, h=self.height, w=self.width, progress=False,
                       n_workers=min(len(self.fns)//10,defaults.cpus))
        self.widget.children = [VBox([
            widgets.Image(value=im.to_bytes_format(), layout=img_layout),
            Dropdown(options=('<Keep>', '<Delete>')+tuple(self.opts), layout={'width': 'max-content'})
        ], layout={'min_width':f'{max(im.width,self.height)}px'}) for im in ims]

    def _ipython_display_(self): display(self.widget)
    def values(self): return L(self.widget.children).itemgot(1).attrgot('value')
    def delete(self): return self.values().argwhere(eq('<Delete>'))
    def change(self): 
        idxs = self.values().argwhere(negate_func(in_(['<Delete>','<Keep>'])))
        return idxs.zipwith(self.values()[idxs])

In [ ]:
fns = get_image_files('images')
w = ImagesCleaner(('A','B'))
w.set_fns(fns)
w

Box(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xbf\x00\x00\x00\x80…

In [ ]:
w.delete(),w.change()

((#1) [0], (#1) [(1, 'A')])

In [ ]:
#export
def _get_iw_info(learn, ds_idx=0):
    dl = learn.dbunch.dls[ds_idx].new(shuffle=False, drop_last=False)
    inp,probs,targs,preds,losses = learn.get_preds(dl=dl, with_input=True, with_loss=True, with_decoded=True)
    inp,targs = L(zip(*dl.decode_batch((inp,targs), max_n=9999)))
    return L([dl.dataset.items,targs,losses]).zip()

In [ ]:
#export
@delegates(ImagesCleaner)
class ImageClassifierCleaner(GetAttr):
    "A widget that provides an `ImagesCleaner` with a CNN `Learner`"
    def __init__(self, learn, **kwargs):
        vocab = learn.dbunch.vocab
        self.default = self.iw = ImagesCleaner(vocab, **kwargs)
        self.dd_cats = Dropdown(options=vocab)
        self.dd_ds   = Dropdown(options=('Train','Valid'))
        self.iwis = _get_iw_info(learn,0),_get_iw_info(learn,1)
        self.dd_ds.observe(self.on_change_ds, 'value')
        self.dd_cats.observe(self.on_change_ds, 'value')
        self.on_change_ds()
        self.widget = VBox([self.dd_cats, self.dd_ds, self.iw.widget])

    def _ipython_display_(self): display(self.widget)
    def on_change_ds(self, change=None):
        info = L(o for o in self.iwis[self.dd_ds.index] if o[1]==self.dd_cats.value)
        self.iw.set_fns(info.sorted(2, reverse=True).itemgot(0))

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.learner.ipynb.
Converted 22_tutorial.imagenette.ipynb.
Converted 23_tutoria